# RNN Language Model

In [94]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_

In [95]:
#https://github.com/pytorch/examples/tree/master/word_language_model 
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)


class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word)  
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

## Hyper-parameters

In [96]:
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [97]:
corpus = Corpus()
ids = corpus.get_data('anna.txt', batch_size)
#ids = corpus.get_data('anna_short.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

## Language Model

In [98]:
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        out = self.linear(out)
        return out, (h, c)

In [99]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers)

In [100]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [101]:
def detach(states):
    return [state.detach() for state in states]

In [102]:
for epoch in range(num_epochs):
    states = ( torch.zeros(num_layers, batch_size, hidden_size),
                    torch.zeros(num_layers, batch_size, hidden_size))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # inputs and targets
        inputs = ids[:, i:i+seq_length]
        targets = ids[:, (i+1):(i+1)+seq_length]
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

Epoch [1/5], Step[0/655], Loss: 10.2829, Perplexity: 29229.76
Epoch [1/5], Step[100/655], Loss: 6.5358, Perplexity: 689.40
Epoch [1/5], Step[200/655], Loss: 6.4094, Perplexity: 607.52
Epoch [1/5], Step[300/655], Loss: 5.9895, Perplexity: 399.23
Epoch [1/5], Step[400/655], Loss: 5.8933, Perplexity: 362.59
Epoch [1/5], Step[500/655], Loss: 5.5209, Perplexity: 249.87
Epoch [1/5], Step[600/655], Loss: 5.4325, Perplexity: 228.72
Epoch [2/5], Step[0/655], Loss: 5.4500, Perplexity: 232.77
Epoch [2/5], Step[100/655], Loss: 5.1026, Perplexity: 164.45
Epoch [2/5], Step[200/655], Loss: 5.0170, Perplexity: 150.95
Epoch [2/5], Step[300/655], Loss: 4.7611, Perplexity: 116.87
Epoch [2/5], Step[400/655], Loss: 4.7792, Perplexity: 119.01
Epoch [2/5], Step[500/655], Loss: 4.4886, Perplexity: 89.00
Epoch [2/5], Step[600/655], Loss: 4.3621, Perplexity: 78.42
Epoch [3/5], Step[0/655], Loss: 4.4412, Perplexity: 84.87
Epoch [3/5], Step[100/655], Loss: 4.1426, Perplexity: 62.97
Epoch [3/5], Step[200/655], Los